In [2]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from keras.layers import LSTM

<h1>Read Datasets<h1>

In [3]:
df = pd.read_excel('Datasets_Ready.xlsx')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 0 to 4509
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      4510 non-null   int64  
 1   Text            4510 non-null   object 
 2   emoji           4510 non-null   object 
 3   Tokenized       4510 non-null   object 
 4   final_text      4494 non-null   object 
 5   text_emoji      4510 non-null   object 
 6   Pos_Word        4510 non-null   int64  
 7   Neg_Word        4510 non-null   int64  
 8   Total_Word      4510 non-null   int64  
 9   Pos_Ratio       4510 non-null   float64
 10  Neg_Ratio       4510 non-null   float64
 11  Sentimen_Text   4510 non-null   object 
 12  Tokenize_Emoji  4510 non-null   object 
 13  Pos_Emoji       4510 non-null   int64  
 14  Neg_Emoji       4510 non-null   int64  
 15  Sentimen_Emoji  4510 non-null   object 
 16  Sarcasm         4510 non-null   object 
dtypes: float64(2), int64(6), object(9

,Unnamed: 0,Text,emoji,Tokenized,final_text,text_emoji,Pos_Word,Neg_Word,Total_Word,Pos_Ratio,Neg_Ratio,Sentimen_Text,Tokenize_Emoji,Pos_Emoji,Neg_Emoji,Sentimen_Emoji,Sarcasm
0,0,kawan2 sunda bersatu jemput arteria dahlan,😁,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",kawan sunda bersatu jemput arteria dahlan,kawan sunda bersatu jemput arteria dahlan 😁,2,0,6,0.333333,0.000000,Positif,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",1,0,Positif,Negatif
1,2,arteria dahlan disidang adat sunda daerah menu...,😁,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",arteria dahlan sidang adat sunda daerah menunt...,arteria dahlan sidang adat sunda daerah menunt...,5,4,9,0.555556,0.444444,Positif,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",1,0,Positif,Negatif
2,4,ga ditafsirkan membanding bandingkan edy arter...,😁,"['ga', 'tafsir', 'membanding', 'membandingkan'...",ga tafsir membanding membandingkan edy arteria...,ga tafsir membanding membandingkan edy arteria...,3,2,9,0.333333,0.222222,Positif,"['ga', 'tafsir', 'membanding', 'membandingkan'...",1,0,Positif,Negatif
3,5,nantik ngak divhumas polri nya bilang palsu td...,😁,"['nanti', 'tidak', 'divisi humas', 'polri', '....",divisi humas polri bilang palsu mengeluarkan a...,divisi humas polri bilang palsu mengeluarkan a...,4,8,16,0.250000,0.500000,Negatif,"['divisi', 'humas', 'polri', 'bilang', 'palsu'...",1,0,Positif,Positif
4,6,urut dada dech kuasa hukum tdk mengerti hukum ...,😁,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'tid...",urut dada deh kuasa hukum mengerti hukum arter...,urut dada deh kuasa hukum mengerti hukum arter...,14,6,28,0.500000,0.214286,Positif,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'men...",1,0,Positif,Negatif


In [4]:
# Remove unused column/feature
df = df.drop(columns='Unnamed: 0')
df = df.drop(columns='Text')
df = df.drop(columns='emoji')
df = df.drop(columns='Tokenized')
df = df.drop(columns='final_text')
df = df.drop(columns='Pos_Word')
df = df.drop(columns='Neg_Word')
df = df.drop(columns='Total_Word')
df = df.drop(columns='Pos_Ratio')
df = df.drop(columns='Neg_Ratio')
df = df.drop(columns='Tokenize_Emoji')
df = df.drop(columns='Pos_Emoji')
df = df.drop(columns='Neg_Emoji')
df = df.drop(columns='Sentimen_Emoji')

In [5]:
df.head()

,text_emoji,Sentimen_Text,Sarcasm
0,kawan sunda bersatu jemput arteria dahlan 😁,Positif,Negatif
1,arteria dahlan sidang adat sunda daerah menunt...,Positif,Negatif
2,ga tafsir membanding membandingkan edy arteria...,Positif,Negatif
3,divisi humas polri bilang palsu mengeluarkan a...,Negatif,Positif
4,urut dada deh kuasa hukum mengerti hukum arter...,Positif,Negatif


In [6]:
print(f'Total Duplicated Data: {df.duplicated().sum()}')

Total Duplicated Data: 120


In [8]:
df = df.drop_duplicates(ignore_index = True)

In [9]:
print(f'Total Duplicated Data After Handling: {df.duplicated().sum()}')

Total Duplicated Data After Handling: 0


In [10]:
print(f'Total Missing Value: \n{df.isnull().sum()}')

Total Missing Value: 
text_emoji       0
Sentimen_Text    0
Sarcasm          0
dtype: int64


In [11]:
df['Sentimen_Text'] = df['Sentimen_Text'].replace('Positif', 1)
df['Sentimen_Text'] = df['Sentimen_Text'].replace('Negatif', 0)

df['Sarcasm'] = df['Sarcasm'].replace('Positif', 1)
df['Sarcasm'] = df['Sarcasm'].replace('Negatif', 0)

In [12]:
print(df['Sarcasm'].value_counts())

0    2244
1    2146
Name: Sarcasm, dtype: int64


<h1>Splitting Data</h1>

In [13]:
X = df['text_emoji'].values
y = df['Sarcasm'].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

<h1>Tokenization</h1>

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer

maxlen = 100
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')

tokenizer.fit_on_texts(X_train)

In [16]:
word_index = tokenizer.word_index

In [17]:
word_index

{'<OOV>': 1,
 '👍': 2,
 'luhut': 3,
 '💩': 4,
 '😁': 5,
 'indonesia': 6,
 'cak': 7,
 '😠': 8,
 'puan': 9,
 'orang': 10,
 '💯': 11,
 'nun': 12,
 'jokowi': 13,
 'presiden': 14,
 'binsar': 15,
 '👎': 16,
 'partai': 17,
 '😊': 18,
 'anies': 19,
 'rakyat': 20,
 '😭': 21,
 'maharani': 22,
 'bu': 23,
 '😨': 24,
 'ganjar': 25,
 'menteri': 26,
 'prabowo': 27,
 'pemimpin': 28,
 'semoga': 29,
 'aja': 30,
 'baswedan': 31,
 'ga': 32,
 'negara': 33,
 'masyarakat': 34,
 '😆': 35,
 'anak': 36,
 'calon': 37,
 '2024': 38,
 '💕': 39,
 'jakarta': 40,
 'kalo': 41,
 'bangsa': 42,
 'demokrasi': 43,
 'politik': 44,
 'banget': 45,
 'ketua': 46,
 'perjuangan': 47,
 'lucu': 48,
 '😔': 49,
 'mbak': 50,
 'nama': 51,
 'salah': 52,
 '✌': 53,
 'firaun': 54,
 'kerja': 55,
 'israel': 56,
 'capres': 57,
 'beliau': 58,
 'sih': 59,
 '1': 60,
 'investasi': 61,
 '😴': 62,
 'mulyadi': 63,
 'cocok': 64,
 'program': 65,
 'kadrun': 66,
 'biar': 67,
 'ri': 68,
 'dunia': 69,
 'dg': 70,
 'edi': 71,
 'lu': 72,
 'negeri': 73,
 'pilih': 74,
 'gub

<h1>Sequence</h1>

In [18]:
sequence_train = tokenizer.texts_to_sequences(X_train)
sequence_test = tokenizer.texts_to_sequences(X_test)

<h1>Padding</h1>

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_padded = pad_sequences(sequence_train, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(sequence_test, maxlen=100, padding='post', truncating='post')

In [20]:
X_train_padded[0]

array([ 269,  242,  786,   19,   37,   14,  858,  284,   81,   19,  200,
        713,   17,   17,   29,   17,  465, 3800,    5,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

<h1>Build Matrix Embedding</h1>

In [21]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embedding(file):
    embeddings_index = dict(get_coefs(*i.split(" ")) for i in open(file, encoding='utf-8'))
    
    return embeddings_index

In [22]:
def make_embedding_matrix(embedding, tokenizer, len_voc):
    all_embs = np.stack(embedding.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    # embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    embedding_matrix = np.zeros((len_voc + 1, embed_size))
    
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embedding.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [23]:
glove_50D = load_embedding('Misc/Embeddings/embeddings50D_300E.txt')
glove_100D = load_embedding('Misc/Embeddings/embeddings100D_300E.txt')
glove_150D = load_embedding('Misc/Embeddings/embeddings150D_300E.txt')

<h1>Implement SMOTE</h1>

In [24]:
smote = SMOTE(random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train_padded, y_train)

In [25]:
unique_values, value_counts = np.unique(y_train_resampled, return_counts=True)

In [26]:
for value, count in zip(unique_values, value_counts):
    print(f"{value}: {count}")

0: 2012
1: 2012


In [27]:
embed_matrix_50D = make_embedding_matrix(glove_50D, tokenizer, len(word_index))
embed_matrix_100D = make_embedding_matrix(glove_100D, tokenizer, len(word_index))
embed_matrix_150D = make_embedding_matrix(glove_150D, tokenizer, len(word_index))

C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3378: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
print(f'Dimension matrix 50D: {embed_matrix_50D.shape}\n')
print(f'Dimension matrix 100D : {embed_matrix_100D.shape}\n')
print(f'Dimension matrix 150D : {embed_matrix_150D.shape}\n')

Dimension matrix 50D: (6520, 50)

Dimension matrix 100D : (6520, 100)

Dimension matrix 150D : (6520, 150)



<h1>Create Bi-LSTM Architecture</h1>

In [ ]:
# # Callbacks
# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if(logs.get('accuracy')>=0.8): #logs.get('val_accuracy')>0.8
#       print("\nAkurasi telah mencapai > 75%!")
#       self.model.stop_training = True
# callbacks = myCallback()

<h2>Train Model Bi-LSTM (Parameter embedding_size = 50)</h2>

In [27]:
model_bilstm_embedsize_50 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_50D.shape[0], embed_matrix_50D.shape[1], input_length=maxlen, weights=[embed_matrix_50D], trainable=False),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_bilstm_embedsize_50.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_embedsize_50.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           326000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 64)                12864     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 459,729
Trainable params: 133,729
Non-trainable params: 326,000
____________________________________________

In [28]:
history_bilstm_embedsize_50 = model_bilstm_embedsize_50.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 9s - loss: 0.6508 - accuracy: 0.6118 - val_loss: 0.5890 - val_accuracy: 0.6629 - 9s/epoch - 68ms/step
Epoch 2/10
126/126 - 5s - loss: 0.5622 - accuracy: 0.7090 - val_loss: 0.5507 - val_accuracy: 0.6902 - 5s/epoch - 39ms/step
Epoch 3/10
126/126 - 5s - loss: 0.5394 - accuracy: 0.7259 - val_loss: 0.5518 - val_accuracy: 0.6879 - 5s/epoch - 39ms/step
Epoch 4/10
126/126 - 5s - loss: 0.5258 - accuracy: 0.7331 - val_loss: 0.5217 - val_accuracy: 0.7130 - 5s/epoch - 38ms/step
Epoch 5/10
126/126 - 5s - loss: 0.5106 - accuracy: 0.7393 - val_loss: 0.5269 - val_accuracy: 0.7267 - 5s/epoch - 38ms/step
Epoch 6/10
126/126 - 5s - loss: 0.4962 - accuracy: 0.7495 - val_loss: 0.5236 - val_accuracy: 0.7267 - 5s/epoch - 38ms/step
Epoch 7/10
126/126 - 5s - loss: 0.4842 - accuracy: 0.7589 - val_loss: 0.5227 - val_accuracy: 0.7244 - 5s/epoch - 39ms/step
Epoch 8/10
126/126 - 5s - loss: 0.4792 - accuracy: 0.7587 - val_loss: 0.5135 - val_accuracy: 0.7517 - 5s/epoch - 39ms/step
Epoch 9/10
126/1

<h2>Train Model Bi-LSTM (Parameter embedding_size = 100)</h2>

In [29]:
model_bilstm_embedsize_100 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=False),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_bilstm_embedsize_100.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_embedsize_100.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          652000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              160800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                12864     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 825,729
Trainable params: 173,729
Non-trainable params: 652,000
__________________________________________

In [30]:
history_bilstm_embedsize_100 = model_bilstm_embedsize_100.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 9s - loss: 0.6239 - accuracy: 0.6543 - val_loss: 0.5791 - val_accuracy: 0.6788 - 9s/epoch - 72ms/step
Epoch 2/10
126/126 - 6s - loss: 0.5604 - accuracy: 0.7065 - val_loss: 0.5576 - val_accuracy: 0.6879 - 6s/epoch - 50ms/step
Epoch 3/10
126/126 - 6s - loss: 0.5285 - accuracy: 0.7366 - val_loss: 0.5320 - val_accuracy: 0.7107 - 6s/epoch - 50ms/step
Epoch 4/10
126/126 - 6s - loss: 0.5084 - accuracy: 0.7438 - val_loss: 0.5284 - val_accuracy: 0.7062 - 6s/epoch - 50ms/step
Epoch 5/10
126/126 - 6s - loss: 0.4906 - accuracy: 0.7517 - val_loss: 0.5210 - val_accuracy: 0.7358 - 6s/epoch - 48ms/step
Epoch 6/10
126/126 - 6s - loss: 0.4737 - accuracy: 0.7627 - val_loss: 0.5135 - val_accuracy: 0.7380 - 6s/epoch - 47ms/step
Epoch 7/10
126/126 - 6s - loss: 0.4596 - accuracy: 0.7691 - val_loss: 0.5078 - val_accuracy: 0.7380 - 6s/epoch - 46ms/step
Epoch 8/10
126/126 - 6s - loss: 0.4406 - accuracy: 0.7781 - val_loss: 0.4920 - val_accuracy: 0.7654 - 6s/epoch - 47ms/step
Epoch 9/10
126/1

<h2>Train Model Bi-LSTM (Parameter embedding_size = 150)</h2>

In [31]:
model_bilstm_embedsize_150 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=False),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_bilstm_embedsize_150.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_embedsize_150.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 150)          978000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                12864     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,191,729
Trainable params: 213,729
Non-trainable params: 978,000
________________________________________

In [32]:
history_bilstm_embedsize_150 = model_bilstm_embedsize_150.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 9s - loss: 0.6209 - accuracy: 0.6451 - val_loss: 0.5696 - val_accuracy: 0.6811 - 9s/epoch - 72ms/step
Epoch 2/10
126/126 - 6s - loss: 0.5488 - accuracy: 0.7127 - val_loss: 0.5594 - val_accuracy: 0.6811 - 6s/epoch - 46ms/step
Epoch 3/10
126/126 - 6s - loss: 0.5376 - accuracy: 0.7239 - val_loss: 0.5461 - val_accuracy: 0.6856 - 6s/epoch - 48ms/step
Epoch 4/10
126/126 - 6s - loss: 0.5066 - accuracy: 0.7453 - val_loss: 0.5335 - val_accuracy: 0.7175 - 6s/epoch - 51ms/step
Epoch 5/10
126/126 - 6s - loss: 0.4845 - accuracy: 0.7614 - val_loss: 0.5332 - val_accuracy: 0.7221 - 6s/epoch - 50ms/step
Epoch 6/10
126/126 - 6s - loss: 0.4692 - accuracy: 0.7719 - val_loss: 0.5117 - val_accuracy: 0.7403 - 6s/epoch - 47ms/step
Epoch 7/10
126/126 - 6s - loss: 0.4465 - accuracy: 0.7863 - val_loss: 0.5264 - val_accuracy: 0.7517 - 6s/epoch - 47ms/step
Epoch 8/10
126/126 - 6s - loss: 0.4339 - accuracy: 0.7955 - val_loss: 0.5099 - val_accuracy: 0.7745 - 6s/epoch - 46ms/step
Epoch 9/10
126/1

<h2>Train Model Bi-LSTM (Parameter epochs = 10)</h2>

In [43]:
model_bilstm_epochs_10 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=False),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_bilstm_epochs_10.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_epochs_10.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 100, 150)          978000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 200)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                12864     
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,191,729
Trainable params: 213,729
Non-trainable params: 978,000
________________________________________

In [44]:
history_bilstm_epochs_10 = model_bilstm_epochs_10.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 12s - loss: 0.6233 - accuracy: 0.6412 - val_loss: 0.5892 - val_accuracy: 0.6697 - 12s/epoch - 94ms/step
Epoch 2/10
126/126 - 6s - loss: 0.5469 - accuracy: 0.7259 - val_loss: 0.5453 - val_accuracy: 0.6993 - 6s/epoch - 51ms/step
Epoch 3/10
126/126 - 6s - loss: 0.5169 - accuracy: 0.7403 - val_loss: 0.5254 - val_accuracy: 0.7312 - 6s/epoch - 50ms/step
Epoch 4/10
126/126 - 6s - loss: 0.4980 - accuracy: 0.7532 - val_loss: 0.5342 - val_accuracy: 0.7107 - 6s/epoch - 51ms/step
Epoch 5/10
126/126 - 6s - loss: 0.4805 - accuracy: 0.7619 - val_loss: 0.5231 - val_accuracy: 0.7221 - 6s/epoch - 51ms/step
Epoch 6/10
126/126 - 6s - loss: 0.4623 - accuracy: 0.7781 - val_loss: 0.5032 - val_accuracy: 0.7517 - 6s/epoch - 51ms/step
Epoch 7/10
126/126 - 6s - loss: 0.4448 - accuracy: 0.7791 - val_loss: 0.5124 - val_accuracy: 0.7608 - 6s/epoch - 51ms/step
Epoch 8/10
126/126 - 6s - loss: 0.4200 - accuracy: 0.8009 - val_loss: 0.5238 - val_accuracy: 0.7016 - 6s/epoch - 51ms/step
Epoch 9/10
126

<h2>Train Model Bi-LSTM (Parameter epochs = 50)</h2>

In [45]:
model_bilstm_epochs_50 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=False),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_bilstm_epochs_50.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_epochs_50.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 100, 150)          978000    
                                                                 
 bidirectional_4 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 200)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                12864     
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,191,729
Trainable params: 213,729
Non-trainable params: 978,000
________________________________________

In [46]:
history_bilstm_epochs_50 = model_bilstm_epochs_50.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=50,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/50
126/126 - 9s - loss: 0.6247 - accuracy: 0.6362 - val_loss: 0.5803 - val_accuracy: 0.6720 - 9s/epoch - 74ms/step
Epoch 2/50
126/126 - 6s - loss: 0.5525 - accuracy: 0.7189 - val_loss: 0.5473 - val_accuracy: 0.7016 - 6s/epoch - 47ms/step
Epoch 3/50
126/126 - 6s - loss: 0.5192 - accuracy: 0.7341 - val_loss: 0.5236 - val_accuracy: 0.7358 - 6s/epoch - 47ms/step
Epoch 4/50
126/126 - 6s - loss: 0.5084 - accuracy: 0.7411 - val_loss: 0.5166 - val_accuracy: 0.7449 - 6s/epoch - 48ms/step
Epoch 5/50
126/126 - 6s - loss: 0.4867 - accuracy: 0.7589 - val_loss: 0.5178 - val_accuracy: 0.7289 - 6s/epoch - 48ms/step
Epoch 6/50
126/126 - 6s - loss: 0.4653 - accuracy: 0.7751 - val_loss: 0.5275 - val_accuracy: 0.7267 - 6s/epoch - 48ms/step
Epoch 7/50
126/126 - 6s - loss: 0.4479 - accuracy: 0.7816 - val_loss: 0.4957 - val_accuracy: 0.7813 - 6s/epoch - 48ms/step
Epoch 8/50
126/126 - 6s - loss: 0.4275 - accuracy: 0.7967 - val_loss: 0.5298 - val_accuracy: 0.7039 - 6s/epoch - 48ms/step
Epoch 9/50
126/1

<h2>Train Model Bi-LSTM (Parameter epochs = 100)</h2>

In [47]:
model_bilstm_epochs_100 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=False),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_bilstm_epochs_100.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_epochs_100.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 100, 150)          978000    
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 200)               0         
                                                                 
 dense_20 (Dense)            (None, 64)                12864     
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,191,729
Trainable params: 213,729
Non-trainable params: 978,000
_______________________________________

In [49]:
history_bilstm_epochs_100 = model_bilstm_epochs_100.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=100,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/100
126/126 - 6s - loss: 0.0328 - accuracy: 0.9888 - val_loss: 1.4728 - val_accuracy: 0.7813 - 6s/epoch - 49ms/step
Epoch 2/100
126/126 - 6s - loss: 0.0302 - accuracy: 0.9888 - val_loss: 1.5398 - val_accuracy: 0.7859 - 6s/epoch - 47ms/step
Epoch 3/100
126/126 - 6s - loss: 0.0337 - accuracy: 0.9883 - val_loss: 1.5002 - val_accuracy: 0.7677 - 6s/epoch - 47ms/step
Epoch 4/100
126/126 - 6s - loss: 0.1947 - accuracy: 0.9622 - val_loss: 1.1660 - val_accuracy: 0.7631 - 6s/epoch - 47ms/step
Epoch 5/100
126/126 - 6s - loss: 0.0380 - accuracy: 0.9883 - val_loss: 1.2780 - val_accuracy: 0.7494 - 6s/epoch - 46ms/step
Epoch 6/100
126/126 - 6s - loss: 0.0259 - accuracy: 0.9913 - val_loss: 1.3267 - val_accuracy: 0.7699 - 6s/epoch - 47ms/step
Epoch 7/100
126/126 - 6s - loss: 0.0194 - accuracy: 0.9943 - val_loss: 1.3557 - val_accuracy: 0.7745 - 6s/epoch - 47ms/step
Epoch 8/100
126/126 - 6s - loss: 0.0192 - accuracy: 0.9940 - val_loss: 1.4498 - val_accuracy: 0.7745 - 6s/epoch - 47ms/step
Epoch 9/

<h2>Train Model Bi-LSTM (Parameter Trainable = True)</h2>

In [58]:
model_bilstm_trainable_true = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=True),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_bilstm_trainable_true.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_trainable_true.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 100, 150)          978000    
                                                                 
 bidirectional_7 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 200)               0         
                                                                 
 dense_30 (Dense)            (None, 64)                12864     
                                                                 
 dense_31 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,191,729
Trainable params: 1,191,729
Non-trainable params: 0
___________________________________________

In [59]:
history_bilstm_trainable_true = model_bilstm_trainable_true.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=100,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/100
126/126 - 14s - loss: 0.6250 - accuracy: 0.6404 - val_loss: 0.5800 - val_accuracy: 0.6629 - 14s/epoch - 109ms/step
Epoch 2/100
126/126 - 10s - loss: 0.4781 - accuracy: 0.7599 - val_loss: 0.5707 - val_accuracy: 0.6834 - 10s/epoch - 81ms/step
Epoch 3/100
126/126 - 10s - loss: 0.2839 - accuracy: 0.8834 - val_loss: 0.6072 - val_accuracy: 0.7608 - 10s/epoch - 78ms/step
Epoch 4/100
126/126 - 11s - loss: 0.1311 - accuracy: 0.9565 - val_loss: 0.6602 - val_accuracy: 0.7904 - 11s/epoch - 84ms/step
Epoch 5/100
126/126 - 10s - loss: 0.0492 - accuracy: 0.9863 - val_loss: 0.8245 - val_accuracy: 0.7813 - 10s/epoch - 79ms/step
Epoch 6/100
126/126 - 10s - loss: 0.0301 - accuracy: 0.9913 - val_loss: 1.0459 - val_accuracy: 0.7631 - 10s/epoch - 78ms/step
Epoch 7/100
126/126 - 11s - loss: 0.0174 - accuracy: 0.9940 - val_loss: 1.0414 - val_accuracy: 0.7722 - 11s/epoch - 84ms/step
Epoch 8/100
126/126 - 10s - loss: 0.0116 - accuracy: 0.9968 - val_loss: 1.2222 - val_accuracy: 0.7790 - 10s/epoch - 8

<h2>Train Model Bi-LSTM (Parameter Learning Rate = 0.0001)</h2>

In [62]:
model_bilstm_learningrate_0_0001 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=True),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_bilstm_learningrate_0_0001.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_learningrate_0_0001.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 100, 150)          978000    
                                                                 
 bidirectional_8 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dropout_17 (Dropout)        (None, 200)               0         
                                                                 
 dense_34 (Dense)            (None, 64)                12864     
                                                                 
 dense_35 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,191,729
Trainable params: 1,191,729
Non-trainable params: 0
___________________________________________

In [63]:
history_bilstm_learningrate_0_0001 = model_bilstm_learningrate_0_0001.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=100,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/100
126/126 - 16s - loss: 0.6929 - accuracy: 0.5102 - val_loss: 0.6908 - val_accuracy: 0.5604 - 16s/epoch - 124ms/step
Epoch 2/100
126/126 - 10s - loss: 0.6833 - accuracy: 0.5927 - val_loss: 0.6696 - val_accuracy: 0.6401 - 10s/epoch - 77ms/step
Epoch 3/100
126/126 - 10s - loss: 0.6002 - accuracy: 0.6938 - val_loss: 0.5940 - val_accuracy: 0.6902 - 10s/epoch - 77ms/step
Epoch 4/100
126/126 - 10s - loss: 0.5497 - accuracy: 0.7254 - val_loss: 0.5781 - val_accuracy: 0.7039 - 10s/epoch - 77ms/step
Epoch 5/100
126/126 - 10s - loss: 0.5157 - accuracy: 0.7433 - val_loss: 0.5682 - val_accuracy: 0.6970 - 10s/epoch - 77ms/step
Epoch 6/100
126/126 - 10s - loss: 0.4884 - accuracy: 0.7662 - val_loss: 0.5715 - val_accuracy: 0.6948 - 10s/epoch - 80ms/step
Epoch 7/100
126/126 - 10s - loss: 0.4568 - accuracy: 0.7860 - val_loss: 0.5587 - val_accuracy: 0.6925 - 10s/epoch - 78ms/step
Epoch 8/100
126/126 - 10s - loss: 0.4291 - accuracy: 0.8089 - val_loss: 0.5717 - val_accuracy: 0.6788 - 10s/epoch - 7

<h2>Train Model Bi-LSTM (Parameter Learning Rate = 0.01)</h2>

In [64]:
model_bilstm_learningrate_0_01 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=True),
        tf.keras.layers.Bidirectional(LSTM(100)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model_bilstm_learningrate_0_01.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_learningrate_0_01.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 100, 150)          978000    
                                                                 
 bidirectional_9 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dropout_18 (Dropout)        (None, 200)               0         
                                                                 
 dense_36 (Dense)            (None, 64)                12864     
                                                                 
 dense_37 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,191,729
Trainable params: 1,191,729
Non-trainable params: 0
___________________________________________

In [65]:
history_bilstm_learningrate_0_01 = model_bilstm_learningrate_0_01.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=100,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/100
126/126 - 13s - loss: 0.6134 - accuracy: 0.6426 - val_loss: 0.5275 - val_accuracy: 0.7107 - 13s/epoch - 107ms/step
Epoch 2/100
126/126 - 10s - loss: 0.3524 - accuracy: 0.8479 - val_loss: 0.6054 - val_accuracy: 0.7335 - 10s/epoch - 79ms/step
Epoch 3/100
126/126 - 10s - loss: 0.1413 - accuracy: 0.9525 - val_loss: 0.6050 - val_accuracy: 0.7927 - 10s/epoch - 79ms/step
Epoch 4/100
126/126 - 10s - loss: 0.0498 - accuracy: 0.9833 - val_loss: 0.8117 - val_accuracy: 0.7995 - 10s/epoch - 79ms/step
Epoch 5/100
126/126 - 10s - loss: 0.0329 - accuracy: 0.9876 - val_loss: 1.0075 - val_accuracy: 0.7768 - 10s/epoch - 79ms/step
Epoch 6/100
126/126 - 10s - loss: 0.0166 - accuracy: 0.9940 - val_loss: 1.2391 - val_accuracy: 0.7790 - 10s/epoch - 79ms/step
Epoch 7/100
126/126 - 10s - loss: 0.0060 - accuracy: 0.9978 - val_loss: 1.4305 - val_accuracy: 0.7859 - 10s/epoch - 79ms/step
Epoch 8/100
126/126 - 10s - loss: 0.0034 - accuracy: 0.9985 - val_loss: 1.5401 - val_accuracy: 0.7836 - 10s/epoch - 7

<h2>Train Model Bi-LSTM (Parameter Hidden Size = 128)</h2>

In [29]:
model_bilstm_hidden_128 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=True),
        tf.keras.layers.Bidirectional(LSTM(128)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model_bilstm_hidden_128.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_hidden_128.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 150)          978000    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              285696    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,280,209
Trainable params: 1,280,209
Non-trainable params: 0
______________________________________________

In [30]:
history_bilstm_hidden_128 = model_bilstm_hidden_128.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=100,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/100
126/126 - 16s - loss: 0.6613 - accuracy: 0.5962 - val_loss: 0.5540 - val_accuracy: 0.6993 - 16s/epoch - 130ms/step
Epoch 2/100
126/126 - 12s - loss: 0.4310 - accuracy: 0.8111 - val_loss: 0.4915 - val_accuracy: 0.7631 - 12s/epoch - 99ms/step
Epoch 3/100
126/126 - 12s - loss: 0.2116 - accuracy: 0.9187 - val_loss: 0.5903 - val_accuracy: 0.7608 - 12s/epoch - 99ms/step
Epoch 4/100
126/126 - 13s - loss: 0.0896 - accuracy: 0.9719 - val_loss: 0.6583 - val_accuracy: 0.7517 - 13s/epoch - 100ms/step
Epoch 5/100
126/126 - 13s - loss: 0.0392 - accuracy: 0.9886 - val_loss: 1.2619 - val_accuracy: 0.7198 - 13s/epoch - 104ms/step
Epoch 6/100
126/126 - 13s - loss: 0.0239 - accuracy: 0.9925 - val_loss: 1.3575 - val_accuracy: 0.7426 - 13s/epoch - 104ms/step
Epoch 7/100
126/126 - 14s - loss: 0.0114 - accuracy: 0.9960 - val_loss: 1.4268 - val_accuracy: 0.7608 - 14s/epoch - 113ms/step
Epoch 8/100
126/126 - 14s - loss: 0.0069 - accuracy: 0.9978 - val_loss: 1.6780 - val_accuracy: 0.7494 - 14s/epoch

<h2>Train Model Bi-LSTM (Parameter Hidden Size = 200)</h2>

In [31]:
model_bilstm_hidden_200 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=True),
        tf.keras.layers.Bidirectional(LSTM(200)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model_bilstm_hidden_200.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm_hidden_200.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 150)          978000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 400)              561600    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                25664     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,565,329
Trainable params: 1,565,329
Non-trainable params: 0
____________________________________________

In [32]:
history_bilstm_hidden_128 = model_bilstm_hidden_200.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=100,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/100
126/126 - 27s - loss: 0.7066 - accuracy: 0.5082 - val_loss: 0.6927 - val_accuracy: 0.5285 - 27s/epoch - 210ms/step
Epoch 2/100
126/126 - 20s - loss: 0.6615 - accuracy: 0.5979 - val_loss: 0.6070 - val_accuracy: 0.6651 - 20s/epoch - 159ms/step
Epoch 3/100
126/126 - 19s - loss: 0.4675 - accuracy: 0.7858 - val_loss: 0.5704 - val_accuracy: 0.6948 - 19s/epoch - 151ms/step
Epoch 4/100
126/126 - 22s - loss: 0.2588 - accuracy: 0.8882 - val_loss: 0.6355 - val_accuracy: 0.7426 - 22s/epoch - 174ms/step
Epoch 5/100
126/126 - 21s - loss: 0.1054 - accuracy: 0.9632 - val_loss: 0.5988 - val_accuracy: 0.7631 - 21s/epoch - 163ms/step
Epoch 6/100
126/126 - 21s - loss: 0.0618 - accuracy: 0.9806 - val_loss: 0.7959 - val_accuracy: 0.8200 - 21s/epoch - 163ms/step
Epoch 7/100
126/126 - 21s - loss: 0.0328 - accuracy: 0.9898 - val_loss: 1.1389 - val_accuracy: 0.7677 - 21s/epoch - 165ms/step
Epoch 8/100
126/126 - 21s - loss: 0.0311 - accuracy: 0.9893 - val_loss: 1.0418 - val_accuracy: 0.7358 - 21s/epo

<h1>Create LSTM Architecture</h1>

<h2>Train Model LSTM (Parameter embedding_size = 50)</h2>

In [37]:
model_lstm_embedsize_50 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_50D.shape[0], embed_matrix_50D.shape[1], input_length=maxlen, weights=[embed_matrix_50D], trainable=False),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm_embedsize_50.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_embedsize_50.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 50)           326000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               60400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                6464      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 392,929
Trainable params: 66,929
Non-trainable params: 326,000
_________________________________________________________________


In [38]:
history_lstm_embedsize_50 = model_lstm_embedsize_50.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 6s - loss: 0.6936 - accuracy: 0.5037 - val_loss: 0.6929 - val_accuracy: 0.5285 - 6s/epoch - 48ms/step
Epoch 2/10
126/126 - 4s - loss: 0.6934 - accuracy: 0.4928 - val_loss: 0.6934 - val_accuracy: 0.4715 - 4s/epoch - 31ms/step
Epoch 3/10
126/126 - 4s - loss: 0.6934 - accuracy: 0.5032 - val_loss: 0.6939 - val_accuracy: 0.4715 - 4s/epoch - 32ms/step
Epoch 4/10
126/126 - 4s - loss: 0.6935 - accuracy: 0.4943 - val_loss: 0.6916 - val_accuracy: 0.5285 - 4s/epoch - 31ms/step
Epoch 5/10
126/126 - 4s - loss: 0.6935 - accuracy: 0.4955 - val_loss: 0.6930 - val_accuracy: 0.5285 - 4s/epoch - 31ms/step
Epoch 6/10
126/126 - 4s - loss: 0.6933 - accuracy: 0.4856 - val_loss: 0.6929 - val_accuracy: 0.5285 - 4s/epoch - 31ms/step
Epoch 7/10
126/126 - 4s - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6933 - val_accuracy: 0.4715 - 4s/epoch - 32ms/step
Epoch 8/10
126/126 - 4s - loss: 0.6933 - accuracy: 0.4943 - val_loss: 0.6935 - val_accuracy: 0.4715 - 4s/epoch - 31ms/step
Epoch 9/10
126/1

<h2>Train Model LSTM (Parameter embedding_size = 100)</h2>

In [39]:
model_lstm_embedsize_100 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=False),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm_embedsize_100.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_embedsize_100.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 100)          652000    
                                                                 
 lstm_6 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_12 (Dense)            (None, 64)                6464      
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 738,929
Trainable params: 86,929
Non-trainable params: 652,000
_________________________________________________________________


In [40]:
history_lstm_embedsize_100 = model_lstm_embedsize_100.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10


126/126 - 6s - loss: 0.6936 - accuracy: 0.5010 - val_loss: 0.6932 - val_accuracy: 0.4715 - 6s/epoch - 50ms/step
Epoch 2/10
126/126 - 4s - loss: 0.6934 - accuracy: 0.4995 - val_loss: 0.6936 - val_accuracy: 0.4715 - 4s/epoch - 32ms/step
Epoch 3/10
126/126 - 4s - loss: 0.6932 - accuracy: 0.5055 - val_loss: 0.6930 - val_accuracy: 0.5285 - 4s/epoch - 32ms/step
Epoch 4/10
126/126 - 4s - loss: 0.6933 - accuracy: 0.4990 - val_loss: 0.6934 - val_accuracy: 0.4715 - 4s/epoch - 32ms/step
Epoch 5/10
126/126 - 4s - loss: 0.6933 - accuracy: 0.4948 - val_loss: 0.6929 - val_accuracy: 0.5285 - 4s/epoch - 32ms/step
Epoch 6/10
126/126 - 4s - loss: 0.6933 - accuracy: 0.4893 - val_loss: 0.6935 - val_accuracy: 0.4715 - 4s/epoch - 32ms/step
Epoch 7/10
126/126 - 4s - loss: 0.6932 - accuracy: 0.4978 - val_loss: 0.6931 - val_accuracy: 0.5285 - 4s/epoch - 32ms/step
Epoch 8/10
126/126 - 4s - loss: 0.6932 - accuracy: 0.4943 - val_loss: 0.6932 - val_accuracy: 0.4715 - 4s/epoch - 32ms/step
Epoch 9/10
126/126 - 4s - l

<h2>Train Model LSTM (Parameter embedding_size = 150)</h2>

In [41]:
model_lstm_embedsize_150 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_150D.shape[0], embed_matrix_150D.shape[1], input_length=maxlen, weights=[embed_matrix_150D], trainable=False),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm_embedsize_150.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_embedsize_150.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 150)          978000    
                                                                 
 lstm_7 (LSTM)               (None, 100)               100400    
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_14 (Dense)            (None, 64)                6464      
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,084,929
Trainable params: 106,929
Non-trainable params: 978,000
_________________________________________________________________


In [42]:
history_lstm_embedsize_150 = model_lstm_embedsize_150.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 7s - loss: 0.6935 - accuracy: 0.4958 - val_loss: 0.6932 - val_accuracy: 0.4715 - 7s/epoch - 57ms/step
Epoch 2/10
126/126 - 5s - loss: 0.6933 - accuracy: 0.4893 - val_loss: 0.6940 - val_accuracy: 0.4715 - 5s/epoch - 38ms/step
Epoch 3/10
126/126 - 5s - loss: 0.6936 - accuracy: 0.4928 - val_loss: 0.6931 - val_accuracy: 0.5285 - 5s/epoch - 38ms/step
Epoch 4/10
126/126 - 5s - loss: 0.6932 - accuracy: 0.4898 - val_loss: 0.6932 - val_accuracy: 0.4715 - 5s/epoch - 38ms/step
Epoch 5/10
126/126 - 5s - loss: 0.6932 - accuracy: 0.4804 - val_loss: 0.6933 - val_accuracy: 0.4715 - 5s/epoch - 38ms/step
Epoch 6/10
126/126 - 5s - loss: 0.6932 - accuracy: 0.4918 - val_loss: 0.6932 - val_accuracy: 0.4715 - 5s/epoch - 39ms/step
Epoch 7/10
126/126 - 5s - loss: 0.6932 - accuracy: 0.4876 - val_loss: 0.6931 - val_accuracy: 0.5285 - 5s/epoch - 40ms/step
Epoch 8/10
126/126 - 5s - loss: 0.6932 - accuracy: 0.4881 - val_loss: 0.6931 - val_accuracy: 0.5285 - 5s/epoch - 39ms/step
Epoch 9/10
126/1

<h2>Train Model LSTM (Parameter epochs = 10)</h2>

In [52]:
model_lstm_epochs_10 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=False),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm_epochs_10.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_epochs_10.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 100, 100)          652000    
                                                                 
 lstm_12 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_12 (Dropout)        (None, 100)               0         
                                                                 
 dense_24 (Dense)            (None, 64)                6464      
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                                 
Total params: 738,929
Trainable params: 86,929
Non-trainable params: 652,000
_________________________________________________________________


In [53]:
history_lstm_epochs_10 = model_lstm_epochs_10.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 7s - loss: 0.6935 - accuracy: 0.4853 - val_loss: 0.6933 - val_accuracy: 0.4715 - 7s/epoch - 54ms/step
Epoch 2/10
126/126 - 5s - loss: 0.6944 - accuracy: 0.5129 - val_loss: 0.6926 - val_accuracy: 0.5285 - 5s/epoch - 37ms/step
Epoch 3/10
126/126 - 5s - loss: 0.6935 - accuracy: 0.4918 - val_loss: 0.6932 - val_accuracy: 0.4715 - 5s/epoch - 36ms/step
Epoch 4/10
126/126 - 5s - loss: 0.6933 - accuracy: 0.4990 - val_loss: 0.6940 - val_accuracy: 0.4715 - 5s/epoch - 37ms/step
Epoch 5/10
126/126 - 5s - loss: 0.6933 - accuracy: 0.4983 - val_loss: 0.6934 - val_accuracy: 0.4715 - 5s/epoch - 36ms/step
Epoch 6/10
126/126 - 5s - loss: 0.6934 - accuracy: 0.5017 - val_loss: 0.6935 - val_accuracy: 0.4715 - 5s/epoch - 37ms/step
Epoch 7/10
126/126 - 5s - loss: 0.6932 - accuracy: 0.4973 - val_loss: 0.6923 - val_accuracy: 0.5285 - 5s/epoch - 37ms/step
Epoch 8/10
126/126 - 5s - loss: 0.6935 - accuracy: 0.4958 - val_loss: 0.6933 - val_accuracy: 0.4715 - 5s/epoch - 37ms/step
Epoch 9/10
126/1

<h2>Train Model LSTM (Parameter epochs = 50)</h2>

In [54]:
model_lstm_epochs_50 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=False),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm_epochs_50.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_epochs_50.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 100, 100)          652000    
                                                                 
 lstm_13 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_13 (Dropout)        (None, 100)               0         
                                                                 
 dense_26 (Dense)            (None, 64)                6464      
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 738,929
Trainable params: 86,929
Non-trainable params: 652,000
_________________________________________________________________


In [55]:
history_lstm_epochs_50 = model_lstm_epochs_50.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=50,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/50
126/126 - 7s - loss: 0.6936 - accuracy: 0.4896 - val_loss: 0.6922 - val_accuracy: 0.5285 - 7s/epoch - 53ms/step
Epoch 2/50
126/126 - 4s - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6961 - val_accuracy: 0.4715 - 4s/epoch - 36ms/step
Epoch 3/50
126/126 - 5s - loss: 0.6935 - accuracy: 0.4960 - val_loss: 0.6937 - val_accuracy: 0.4715 - 5s/epoch - 36ms/step
Epoch 4/50
126/126 - 5s - loss: 0.6933 - accuracy: 0.4975 - val_loss: 0.6937 - val_accuracy: 0.4715 - 5s/epoch - 37ms/step
Epoch 5/50
126/126 - 5s - loss: 0.6933 - accuracy: 0.4933 - val_loss: 0.6931 - val_accuracy: 0.5285 - 5s/epoch - 37ms/step
Epoch 6/50
126/126 - 5s - loss: 0.6934 - accuracy: 0.4911 - val_loss: 0.6937 - val_accuracy: 0.4715 - 5s/epoch - 36ms/step
Epoch 7/50
126/126 - 4s - loss: 0.6934 - accuracy: 0.4938 - val_loss: 0.6929 - val_accuracy: 0.5285 - 4s/epoch - 36ms/step
Epoch 8/50
126/126 - 5s - loss: 0.6935 - accuracy: 0.4945 - val_loss: 0.6937 - val_accuracy: 0.4715 - 5s/epoch - 36ms/step
Epoch 9/50
126/1

<h2>Train Model LSTM (Parameter epochs = 100)</h2>

In [56]:
model_lstm_epochs_100 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=False),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm_epochs_100.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_epochs_100.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 100)          652000    
                                                                 
 lstm_14 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_14 (Dropout)        (None, 100)               0         
                                                                 
 dense_28 (Dense)            (None, 64)                6464      
                                                                 
 dense_29 (Dense)            (None, 1)                 65        
                                                                 
Total params: 738,929
Trainable params: 86,929
Non-trainable params: 652,000
_________________________________________________________________


In [57]:
history_lstm_epochs_100 = model_lstm_epochs_100.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=100,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/100
126/126 - 7s - loss: 0.6934 - accuracy: 0.4871 - val_loss: 0.6936 - val_accuracy: 0.4715 - 7s/epoch - 52ms/step
Epoch 2/100
126/126 - 4s - loss: 0.6935 - accuracy: 0.4953 - val_loss: 0.6933 - val_accuracy: 0.4715 - 4s/epoch - 36ms/step
Epoch 3/100
126/126 - 5s - loss: 0.6934 - accuracy: 0.4968 - val_loss: 0.6937 - val_accuracy: 0.4715 - 5s/epoch - 38ms/step
Epoch 4/100
126/126 - 5s - loss: 0.6934 - accuracy: 0.4841 - val_loss: 0.6938 - val_accuracy: 0.4715 - 5s/epoch - 37ms/step
Epoch 5/100
126/126 - 5s - loss: 0.6934 - accuracy: 0.4943 - val_loss: 0.6932 - val_accuracy: 0.4715 - 5s/epoch - 36ms/step
Epoch 6/100
126/126 - 5s - loss: 0.6933 - accuracy: 0.4920 - val_loss: 0.6930 - val_accuracy: 0.5285 - 5s/epoch - 36ms/step
Epoch 7/100
126/126 - 4s - loss: 0.6932 - accuracy: 0.4893 - val_loss: 0.6930 - val_accuracy: 0.5285 - 4s/epoch - 35ms/step
Epoch 8/100
126/126 - 4s - loss: 0.6932 - accuracy: 0.4933 - val_loss: 0.6931 - val_accuracy: 0.5285 - 4s/epoch - 35ms/step
Epoch 9/

<h2>Train Model LSTM (Parameter Trainable = True)</h2>

In [60]:
model_lstm_trainable_true = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=True),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm_trainable_true.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_trainable_true.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 100, 100)          652000    
                                                                 
 lstm_16 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 dense_32 (Dense)            (None, 64)                6464      
                                                                 
 dense_33 (Dense)            (None, 1)                 65        
                                                                 
Total params: 738,929
Trainable params: 738,929
Non-trainable params: 0
_________________________________________________________________


In [61]:
history_lstm_trainable_true = model_lstm_trainable_true.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=50,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/50
126/126 - 9s - loss: 0.6938 - accuracy: 0.4829 - val_loss: 0.6939 - val_accuracy: 0.4715 - 9s/epoch - 72ms/step
Epoch 2/50
126/126 - 7s - loss: 0.6936 - accuracy: 0.4993 - val_loss: 0.6953 - val_accuracy: 0.4715 - 7s/epoch - 53ms/step
Epoch 3/50
126/126 - 7s - loss: 0.6933 - accuracy: 0.5025 - val_loss: 0.6915 - val_accuracy: 0.5285 - 7s/epoch - 53ms/step
Epoch 4/50
126/126 - 7s - loss: 0.6937 - accuracy: 0.4935 - val_loss: 0.6926 - val_accuracy: 0.5285 - 7s/epoch - 53ms/step
Epoch 5/50
126/126 - 7s - loss: 0.6935 - accuracy: 0.4876 - val_loss: 0.6930 - val_accuracy: 0.5285 - 7s/epoch - 53ms/step
Epoch 6/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.5080 - val_loss: 0.6923 - val_accuracy: 0.5285 - 7s/epoch - 54ms/step
Epoch 7/50
126/126 - 7s - loss: 0.6936 - accuracy: 0.5017 - val_loss: 0.6928 - val_accuracy: 0.5285 - 7s/epoch - 55ms/step
Epoch 8/50
126/126 - 7s - loss: 0.6935 - accuracy: 0.4938 - val_loss: 0.6928 - val_accuracy: 0.5285 - 7s/epoch - 54ms/step
Epoch 9/50
126/1

<h2>Train Model LSTM (Parameter Learning Rate = 0.0001)</h2>

In [66]:
model_lstm_learningrate_0_0001 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=True),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_lstm_learningrate_0_0001.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_learningrate_0_0001.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 100, 100)          652000    
                                                                 
 lstm_19 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_19 (Dropout)        (None, 100)               0         
                                                                 
 dense_38 (Dense)            (None, 64)                6464      
                                                                 
 dense_39 (Dense)            (None, 1)                 65        
                                                                 
Total params: 738,929
Trainable params: 738,929
Non-trainable params: 0
_________________________________________________________________


In [67]:
history_lstm_learningrate_0_0001 = model_lstm_learningrate_0_0001.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=50,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/50
126/126 - 9s - loss: 0.6932 - accuracy: 0.4918 - val_loss: 0.6933 - val_accuracy: 0.4715 - 9s/epoch - 75ms/step
Epoch 2/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.4958 - val_loss: 0.6933 - val_accuracy: 0.4715 - 7s/epoch - 56ms/step
Epoch 3/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.4863 - val_loss: 0.6932 - val_accuracy: 0.4715 - 7s/epoch - 57ms/step
Epoch 4/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.5002 - val_loss: 0.6932 - val_accuracy: 0.4715 - 7s/epoch - 57ms/step
Epoch 5/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.4903 - val_loss: 0.6932 - val_accuracy: 0.4715 - 7s/epoch - 57ms/step
Epoch 6/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.5037 - val_loss: 0.6931 - val_accuracy: 0.5285 - 7s/epoch - 57ms/step
Epoch 7/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.4920 - val_loss: 0.6933 - val_accuracy: 0.4715 - 7s/epoch - 57ms/step
Epoch 8/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6933 - val_accuracy: 0.4715 - 7s/epoch - 57ms/step
Epoch 9/50
126/1

<h2>Train Model LSTM (Parameter Learning Rate = 0.01)</h2>

In [68]:
model_lstm_learningrate_0_01 = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(embed_matrix_100D.shape[0], embed_matrix_100D.shape[1], input_length=maxlen, weights=[embed_matrix_100D], trainable=True),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model_lstm_learningrate_0_01.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm_learningrate_0_01.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 100, 100)          652000    
                                                                 
 lstm_20 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_20 (Dropout)        (None, 100)               0         
                                                                 
 dense_40 (Dense)            (None, 64)                6464      
                                                                 
 dense_41 (Dense)            (None, 1)                 65        
                                                                 
Total params: 738,929
Trainable params: 738,929
Non-trainable params: 0
_________________________________________________________________


In [69]:
history_lstm_learningrate_0_01 = model_lstm_learningrate_0_01.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=50,
        validation_data=(X_test_padded, y_test),
        verbose=2
)

Epoch 1/50
126/126 - 9s - loss: 0.6961 - accuracy: 0.4940 - val_loss: 0.6918 - val_accuracy: 0.5285 - 9s/epoch - 73ms/step
Epoch 2/50
126/126 - 8s - loss: 0.6938 - accuracy: 0.4863 - val_loss: 0.6928 - val_accuracy: 0.5285 - 8s/epoch - 62ms/step
Epoch 3/50
126/126 - 7s - loss: 0.6934 - accuracy: 0.4896 - val_loss: 0.6929 - val_accuracy: 0.5285 - 7s/epoch - 56ms/step
Epoch 4/50
126/126 - 7s - loss: 0.6936 - accuracy: 0.4846 - val_loss: 0.6931 - val_accuracy: 0.5285 - 7s/epoch - 58ms/step
Epoch 5/50
126/126 - 7s - loss: 0.6935 - accuracy: 0.4970 - val_loss: 0.6928 - val_accuracy: 0.5285 - 7s/epoch - 58ms/step
Epoch 6/50
126/126 - 7s - loss: 0.6934 - accuracy: 0.4881 - val_loss: 0.6940 - val_accuracy: 0.4715 - 7s/epoch - 55ms/step
Epoch 7/50
126/126 - 7s - loss: 0.6940 - accuracy: 0.4965 - val_loss: 0.6932 - val_accuracy: 0.4715 - 7s/epoch - 58ms/step
Epoch 8/50
126/126 - 7s - loss: 0.6932 - accuracy: 0.5117 - val_loss: 0.6957 - val_accuracy: 0.4715 - 7s/epoch - 56ms/step
Epoch 9/50
126/1

<h1>Create SVM Model</h1>

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

In [34]:
vectorizer = CountVectorizer(binary=True)

vectorizer.fit(list(X_train) + list(X_test))

X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [35]:
svc = svm.SVC(kernel='linear', probability=True)

prob = svc.fit(X_train_vec, y_train).predict_proba(X_test_vec)

y_pred_svm = svc.predict(X_test_vec)

In [38]:
print(f'Accuracy : {round(accuracy_score(y_test, y_pred_svm) * 100, 2)}%')

Accuracy : 54.21%


In [39]:
prob

array([[0.51800326, 0.48199674],
       [0.52244987, 0.47755013],
       [0.56833728, 0.43166272],
       [0.4204815 , 0.5795185 ],
       [0.5212829 , 0.4787171 ],
       [0.51432481, 0.48567519],
       [0.51902092, 0.48097908],
       [0.56711882, 0.43288118],
       [0.49446049, 0.50553951],
       [0.5       , 0.5       ],
       [0.47902939, 0.52097061],
       [0.5       , 0.5       ],
       [0.55123352, 0.44876648],
       [0.47908495, 0.52091505],
       [0.50662938, 0.49337062],
       [0.46641028, 0.53358972],
       [0.44842109, 0.55157891],
       [0.53769802, 0.46230198],
       [0.52303735, 0.47696265],
       [0.52244987, 0.47755013],
       [0.52554246, 0.47445754],
       [0.55503394, 0.44496606],
       [0.5       , 0.5       ],
       [0.51511704, 0.48488296],
       [0.52119019, 0.47880981],
       [0.52018892, 0.47981108],
       [0.49465392, 0.50534608],
       [0.5       , 0.5       ],
       [0.51083051, 0.48916949],
       [0.53772499, 0.46227501],
       [0.